In [ ]:
# Импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Загрузка датафрейма
df = pd.read_csv('data_after_eda (2).csv')

In [ ]:
# Задание целевой переменной worldwide_gross (кассовые сборы)
y = df["worldwide_gross"]
X = df.drop(columns=[
    "worldwide_gross", "Unnamed: 0", "title", "writer", "cast"
])

# Определение цифровых и категориальных переменных
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "profit_or_loss", "p&l_rate",
                    "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Обработка категориальных переменных
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Построение модели линейной регрессии
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

MSE: 0.00
R^2 Score: 1.00


Перефит: модель слишком хорошо подогналась под тренировочные данные, причиной могло послужить "утечка данных" (например, фичи напрямую коррелируют с worldwide_gross - profit_or_loss).


In [ ]:
# Загрузка датафрейма
df_1 = pd.read_csv('data_after_eda (2).csv')

In [ ]:
# Задание целевой переменной worldwide_gross (кассовые сборы),
# Устранение целевых признаков утечки (p&l_rate)
df_1 = df_1.drop(columns=["profit_or_loss", "p&l_rate"])

# Задание целевой функции и переменной
y = df_1["worldwide_gross"]
X = df_1.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast"])

#  Определение цифровых и категориальных переменных
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Разделение категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Построение модели ридж-регрессии
model_2 = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_2.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = model_2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

MSE: 18899663850822424.00
R^2 Score: 0.61


Итак, коэффициент детерминации R^2 модели линейной регрессии равен 0,61, но величина MSE - огромна.

Попробуйем протестировать другую модель, например, ридж-регрессии.

In [ ]:
# Загрузка датафрейма
df_2 = pd.read_csv('data_after_eda (2).csv')

In [ ]:
# Задание целевой переменной worldwide_gross (кассовые сборы),
# Устранение целевых признаков утечки (p&l_rate)
df_2 = df_2.drop(columns=["profit_or_loss", "p&l_rate"])

# Задание целевой функции и переменной
y = df_2["worldwide_gross"]
X = df_2.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast"])

#  Определение цифровых и категориальных переменных
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Разделение категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Построение модели ридж-регрессии
ridge_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0))
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
ridge_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Ridge Regression Results:")
print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Ridge Regression Results:
MSE: 324124552742943522816.00
R^2 Score: -6742.36


Модель Ridge Regression отработала плохо. Далее нет смысла ее улучшать.

Попробуем протестировать модель Lasso-регрессии. Данная модель проводит отбор признаков.

In [ ]:
from sklearn.linear_model import LassoCV

# Построение модели
lasso_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LassoCV(cv=5))
])

# Обучение модели
lasso_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\nLasso Regression Results:")
print(f"MSE: {mse_lasso:.2f}")
print(f"R^2 Score: {r2_lasso:.2f}")



Lasso Regression Results:
MSE: 18853453312172448.00
R^2 Score: 0.61


Модель Lasso Regression успешно провела регуляризацию и отброс лишние фичи, справившись с переобучением. При этом коэффициент детерминации R^2 модели  Lasso Regression равен 0,61 (как и у линейной регрессии). И при этом значение MSE также сликом большое.

Возможно, призанки надо было отмасштабировать. Сделаем это в следующем шаге для модели Lasso Regression.

In [ ]:
# Построение модели с масштабированием
lasso_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LassoCV(cv=5, max_iter=10000))
])

# Обучение модели
lasso_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\nLasso Regression Results:")
print(f"MSE: {mse_lasso:.2f}")
print(f"R^2 Score: {r2_lasso:.2f}")


Lasso Regression Results:
MSE: 18853453312172448.00
R^2 Score: 0.61


Масштабирование не повлияло на R^2, так как она уже сходилась корректно. Дело в том, что Lasso автоматически отбирает признаки, зануляя коэффициенты некоторых из них.

Попробуем протестировать модель Kernel Ridge Regression. Она исполтзуется для моделирования нелинейных зависимостей между признаками и целевой переменной.

In [ ]:
# Импорт библиотеки
from sklearn.kernel_ridge import KernelRidge

# Загрузка датасета, подготовка данных
df_3 = pd.read_csv('data_after_eda (2).csv')
df_3 = df_3.drop(columns=["profit_or_loss", "p&l_rate"])
y = df_3["worldwide_gross"]
X = df_3.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast"])

# Разделение числовых и категориальных признаков
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Построение модели
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Обучение модели
kernel_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", KernelRidge(kernel='rbf', alpha=1.0))
])

# Прогноз и оценка модели
kernel_model.fit(X_train, y_train)
y_pred_kernel = kernel_model.predict(X_test)
print("\nKernel Ridge Regression Results:")
print("MSE:", mean_squared_error(y_test, y_pred_kernel))
print("R^2:", r2_score(y_test, y_pred_kernel))



Kernel Ridge Regression Results:
MSE: 2.0829011516680348e+16
R^2: 0.5666558771841989


KernelRidge работает немного хуже, чем Lasso и линейная регрессия (0,56 против 0,61), но лучше, чем Ridge. Но MSE - гигантское.

Попробуем улучшить модель KernelRidge, произведя автоматический подбор наилучших гиперпараметров модели с помощью перебора по сетке и кросс-валидации.
То есть перебираются значения alpha (регуляризация) и gamma (ширина ядра RBF). В данном случае:
cv=5: 5-кратная кросс-валидация;
scoring='r2': метрика для оценки модели;
n_jobs=-1: использовать все ядра CPU.

In [ ]:
# Загрузка датасета, подготовка данных
df_4 = pd.read_csv('data_after_eda (2).csv')
df_4 = df_4.drop(columns=["profit_or_loss", "p&l_rate"])
y = df_4["worldwide_gross"]
X = df_4.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast"])

# Разделение числовых и категориальных признаков
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Построение модели
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели (с автоподбором параметров)
krr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", KernelRidge(kernel='rbf'))
])

# Начальные настройки для автоподбора гиперпараметров
grid_params = {
    "regressor__alpha": [0.1, 1.0, 10.0, 100.0],
    "regressor__gamma": [1e-3, 1e-2, 1e-1, 1.0]
}

# Автоподбор гиперпараметров
search = GridSearchCV(krr_pipeline, grid_params, cv=5, scoring='r2', verbose=1, n_jobs=-1)
search.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = search.predict(X_test)
print("Best Kernel Ridge Parameters:", search.best_params_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R^2:", r2_score(y_test, y_pred))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Kernel Ridge Parameters: {'regressor__alpha': 0.1, 'regressor__gamma': 0.01}
MSE: 1.755824283589858e+16
R^2: 0.6347036759850065


KernelRidge с автоматическим подбором наилучших гиперпараметров показала R^2 равный 0,63, теперь она немного лучше, чем Lasso-регрессия и классическая регрессия.

Попробуем протестировать модель CatBoostRegressor (которая работает с сырыми данными).

In [ ]:
# Загрузка датасета, подготовка данных
df_5 = pd.read_csv("data_after_eda (2).csv")
df_5 = df_5.drop(columns=["profit_or_loss", "p&l_rate"])
y = df_5["worldwide_gross"]
X = df_5.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast"])

# Разделение числовых и категориальных признаков
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]
categorical_features = ["director", "country_of_origin", "languages", "genre"]

# Построение модели
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Фиксация параметров для последующих преобразований
preprocessor.fit(X_train)

# Построение модели
catboost_model = CatBoostRegressor(verbose=0, random_state=42)

# Обучение модели
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Прогноз и оценка модели
catboost_model.fit(X_train_processed, y_train)
y_pred_cb = catboost_model.predict(X_test_processed)
print("\nCatBoost Regressor:")
print("MSE:", mean_squared_error(y_test, y_pred_cb))
print("R^2:", r2_score(y_test, y_pred_cb))


CatBoost Regressor:
MSE: 1.4545365258669544e+16
R^2: 0.697386093237988


CatBoostRegressor показывает лучший результат R^2: 0,70. Значит, модель все-таки нелинейна (это логично, с учетом достаточного большого количества категориальных признаков).

А что будет, если все-таки удалить все категориальные признаки?

In [ ]:
# Загрузка датасета, подготовка данных
df_6 = pd.read_csv("data_after_eda (2).csv")
df_6 = df_6.drop(columns=["profit_or_loss", "p&l_rate"])
y = df_6["worldwide_gross"]
X = df_6.drop(columns=["worldwide_gross", "Unnamed: 0", "title", "writer", "cast", "director", "country_of_origin", "languages", "genre"])

Классическая модель регрессии

In [ ]:
#  Определение цифровых и категориальных переменных
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]

# Разделение категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
    ]
)

# Построение модели ридж-регрессии
model_2 = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_2.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = model_2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

MSE: 17862038853059194.00
R^2 Score: 0.63


Ридж-регрессия

In [ ]:
#  Определение цифровых и категориальных переменных
numeric_features = ["average_rating", "metascore", "runtime", "budget",
                    "release_year", "lang_cnt", "country_cnt", "coproduction"]

# Разделение категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
])

# Построение модели ридж-регрессии
from sklearn.linear_model import Ridge
ridge_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0))
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
ridge_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Ridge Regression Results:")
print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")
from sklearn.linear_model import Ridge
ridge_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0))
])

# Разделение на тестовую 80% и тренировочную 20% выборку (с рандомным коэффициентом 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
ridge_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Ridge Regression Results:")
print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Ridge Regression Results:
MSE: 17864991229121036.00
R^2 Score: 0.63
Ridge Regression Results:
MSE: 17864991229121036.00
R^2 Score: 0.63


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.66767e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.66767e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Lasso-регрессия

In [ ]:
# Построение модели
lasso_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LassoCV(cv=5))
])

# Обучение модели
lasso_model.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\nLasso Regression Results:")
print(f"MSE: {mse_lasso:.2f}")
print(f"R^2 Score: {r2_lasso:.2f}")


Lasso Regression Results:
MSE: 18853453312172468.00
R^2 Score: 0.61


KernelRidge регрессия

In [ ]:
# Обучение модели
kernel_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", KernelRidge(kernel='rbf', alpha=1.0))
])

# Прогноз и оценка модели
kernel_model.fit(X_train, y_train)
y_pred_kernel = kernel_model.predict(X_test)
print("\nKernel Ridge Regression Results:")
print("MSE:", mean_squared_error(y_test, y_pred_kernel))
print("R^2:", r2_score(y_test, y_pred_kernel))


Kernel Ridge Regression Results:
MSE: 6.348504630773886e+16
R^2: -0.32079583719641613



KernelRidge регрессия с автоподбором параметров

In [ ]:
# Обучение модели (с автоподбором параметров)
krr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", KernelRidge(kernel='rbf'))
])

# Начальные настройки для автоподбора гиперпараметров
grid_params = {
    "regressor__alpha": [0.1, 1.0, 10.0, 100.0],
    "regressor__gamma": [1e-3, 1e-2, 1e-1, 1.0]
}

# Автоподбор гиперпараметров
search = GridSearchCV(krr_pipeline, grid_params, cv=5, scoring='r2', verbose=1, n_jobs=-1)
search.fit(X_train, y_train)

# Прогноз и оценка модели
y_pred = search.predict(X_test)
print("Best Kernel Ridge Parameters:", search.best_params_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R^2:", r2_score(y_test, y_pred))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Kernel Ridge Parameters: {'regressor__alpha': 0.1, 'regressor__gamma': 0.001}
MSE: 2.8941845126541864e+16
R^2: 0.39786972228671


KernelRidge отработала плохо